## Step 1: Get Top Songs by Artist

**API Endpoint:** Last.fm `artist.getTopTracks`

**Base URL:** `http://ws.audioscrobbler.com/2.0/`

**Parameters:** `artist`, `api_key`, `format`, `limit`

In [1]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

params = {
    'method': 'artist.getTopTracks',
    'artist': 'Taylor Swift',
    'api_key': api_key,
    'format': 'json',
    'limit': 10
}

response = requests.get('http://ws.audioscrobbler.com/2.0/', params=params)

if response.status_code == 200:
    data = response.json()
    if 'toptracks' in data:
        print(f"Top 10 songs by '{params['artist']}':\n")
        for i, track in enumerate(data['toptracks']['track'], 1):
            print(f"{i}. {track['name']} - {int(track['playcount']):,} plays")


Top 10 songs by 'Taylor Swift':

1. Blank Space - 27,308,496 plays
2. Cruel Summer - 48,068,549 plays
3. Shake It Off - 18,369,691 plays
4. Style - 32,427,431 plays
5. cardigan - 43,802,061 plays
6. Love Story - 14,095,205 plays
7. You Belong with Me - 13,364,844 plays
8. august - 41,913,851 plays
9. Anti-Hero - 33,905,103 plays
10. Lover - 26,892,624 plays


## Step 2: Sort and Display Top 3 Most Popular Songs

This section demonstrates how to:
- Sort the retrieved tracks by playcount (popularity)
- Extract and display only the top 3 most popular songs

In [2]:
import pandas as pd

if 'data' in locals() and 'toptracks' in data:
    df = pd.DataFrame(data['toptracks']['track'])[['name', 'playcount']]
    df['playcount'] = pd.to_numeric(df['playcount'], errors='coerce').astype(int)
    top3 = df.sort_values('playcount', ascending=False).head(3)
    
    print("Top 3 most popular songs:\n")
    for i, (_, row) in enumerate(top3.iterrows(), 1):
        print(f"{i}. {row['name']} - {row['playcount']:,} plays")


Top 3 most popular songs:

1. Cruel Summer - 48,068,549 plays
2. cardigan - 43,802,061 plays
3. august - 41,913,851 plays


## Step 3: Get Top Tags for Each Track

This section demonstrates how to:
- Fetch top tags/genres for each track using the `track.getTopTags` API method
- Create a reusable function to retrieve tags for any track
- Add a new column to the DataFrame containing the top 5 tags for each song

In [3]:
import requests
import pandas as pd

def get_top_tags(artist, track_name, max_tags=5):
    response = requests.get('http://ws.audioscrobbler.com/2.0/', params={
        'method': 'track.getTopTags', 'artist': artist, 'track': track_name,
        'api_key': api_key, 'format': 'json'
    })
    tags = response.json().get('toptags', {}).get('tag', [])
    return [tag['name'] for tag in tags[:max_tags]] if response.status_code == 200 else []

if 'df' in locals():
    df['top_tags'] = [get_top_tags(params['artist'], name) for name in df['name']]
    print(df[['name', 'playcount', 'top_tags']].head())


           name  playcount                                           top_tags
0   Blank Space   27308496  [pop, love at first listen, 2014, female vocal...
1  Cruel Summer   48068549  [electropop, pop, synthpop, industrial pop, sy...
2  Shake It Off   18369691  [pop, 2014, haters gonna hate, taylor swift, l...
3         Style   32427431  [pop, synthpop, taylor swift, love at first li...
4      cardigan   43802061  [folk pop, folk, chamber pop, singer-songwrite...
